# Electric Vehicle Example

## Target EV model

A simple 1D EV model, consisting of a battery, a motor, and tire.  Each of them has a `#simBlock action` (inheriting `StateSpaceRepresentation::ContinuousStateSpaceDynamics`), which corresponds to a Simulink block.

Note that attribute values marked with `@MgS::Param` are reflected to the model workspace of Simulink.

In [2]:
%load EV2.sysml

Loaded EV2.sysml


In [4]:
%edit EV2.sysml

Edit EV2.sysml 
 
failed to open EV2.sysml

In [ ]:
%viz --view tree --style comptree --style showinherited --style stdcolor --style lr EV2::Vehicle

## Compact and Large EV Contexts

In [ ]:
package CompactEV {
    import EV2::*;

    part vehicle_compact :> baseVehicle {
        :>> mass = 800[kg];
        action :>> battery {
            :>> capacity = 50['A⋅h'];
        }
        
        action :>> powerTrain {
            action motor : EVlib::Motor {
                :>> torqueCoeff = 10 ['N⋅m/A'];
                :>> motR = 4['Ω'];
                :>> motL = 0.2[H];
            }
            action tire : EVlib::Tire {
                :>> moment = 200['kg⋅m²'];
                :>> radius = 0.5[m];
            }
            bind input.voltage = motor.input.voltage;
            bind motor.output.current = output.current;

            flow mt2tt from motor.output.torque to tire.input.torque;
            flow to2mf from tire.output.torque to motor.input.friction;
            
            bind input.accel = tire.input.accel;
            bind tire.output.force = output.force;
        }
    }

    part smallEVRangeContext {
        requirement smallEVRequirement : VehicleRequirement {
            doc /* The small EVs must be ligher than 900[kg] */
            in :>> vehicle = vehicle_compact;
            /*  To comform with the regulation and the battery mass will impact it. */
            assume constraint { vehicle.mass < 900[kg] }
        }

        analysis smallEVAnalysis : VehicleAnalysis {
            subject :>> vehicle :> vehicle_compact;
            requirement :>> vehicleRequirement = smallEVRequirement;
        }

        requirement <C1> rangeRequirementSmall :> smallEVRequirement : RangeRequirement {
            doc /* The small EVs must run longer than 130km */
            attribute :>> requiredRange = 130[km];
        }

        analysis rangeAnalysisSmall :> smallEVAnalysis : RangeAnalysis {
            :>> simCond { :>> toTime = 5000[s]; }
            requirement :>> rangeRequirement = rangeRequirementSmall;
            return simulatedRange = vehicle.vehicleBehavior.output.distance {
                @MgS::SimRet {
                    ret = MgS::Retrieval::FINAL;
                }
            }
        }

        requirement <C2> efficiencyRequirementSmall :> smallEVRequirement : EfficiencyRequirement {
            doc /* The target efficiency of small EVs is 0.9. */
            attribute :>> requiredEfficiency = 0.9;
        }

        analysis efficiencyAnalysisSmall :> smallEVAnalysis : EfficiencyAnalysis {
            :>> simCond { :>> toTime = 1800[s]; }
            requirement :>> efficiencyRequirement = efficiencyRequirementSmall;

            return simulatedEfficiency = vehicle.efficiency { @MgS::SimRet; }
        }

        requirement <C3> maxSpeedRequirementSmall :> smallEVRequirement : MaxSpeedRequirement {
            doc /* The target maximum speed of small EVs is 130 [km/h]. */
            attribute :>> requiredMaxSpeed = 130 [km/h];
        }

        analysis maxSpeedAnalysisSmall :> smallEVAnalysis : MaxSpeedAnalysis {
            :>> simCond { :>> toTime = 400[s]; }
            requirement :>> maxSpeedRequirement = maxSpeedRequirementSmall;
            out voltage :> ISQ::electricPotential = vehicle.battery.output.voltage {
                @MgS::SimRet {
                    ret = Retrieval::FULL;
                }
            }
            return simulatedMaxSpeed = vehicle.vehicleBehavior.output.velocity {
                @MgS::SimRet {
                    ret = Retrieval::MAX;
                }
            }
        }
    }
}

In [ ]:
%viz --view tree --style stdcolor --style lr CompactEV::smallEVRangeContext

In [ ]:
%viz --view action --style lr --style ortholine --style stdcolor CompactEV::vehicle_compact

In [3]:
package LargeEV {
    import EV2::*;
    
    part vehicle_large :> baseVehicle {
        :>> mass = 1100[kg];
        action :>> battery {
            :>> capacity = 70['A⋅h'];
        }
        
        attribute baseResistance = 3['Ω'];
        action :>> powerTrain {
            action motor : EVlib::Motor {
                :>> torqueCoeff = 13 ['N⋅m/A'];
                :>> motR = 5['Ω'] + baseResistance;
                :>> motL = 0.2[H];
            }
            action tire : EVlib::Tire {
                :>> moment = 300['kg⋅m²'];
                :>> radius = 0.7[m];
            }
            bind input.voltage = motor.input.voltage;
            bind motor.output.current = output.current;

            flow mt2rt from motor.output.torque to reducer.input.torque;
            
            action reducer : EVlib::Reducer {
                :>> gearRatio = 1.5;
            }
            
            flow rt2tt from reducer.output.torque to tire.input.torque;
            flow to2mf from tire.output.torque to motor.input.friction;
            
            bind input.accel = tire.input.accel;
            bind tire.output.force = output.force;
        }
        
    }

    part largeEVRangeContext {
        requirement largeEVRequirement : VehicleRequirement {
            doc /* The large EVs must be ligher than 900[kg] */
            in :>> vehicle = vehicle_large;
            /*  To comform with the regulation and the battery mass will impact it. */
            assume constraint { vehicle.mass < 1200[kg] }
        }

        analysis largeEVAnalysis : VehicleAnalysis {
            subject :>> vehicle :> vehicle_large;
            requirement :>> vehicleRequirement = largeEVRequirement;
        }

        requirement <L1> rangeRequirementLarge :> largeEVRequirement : RangeRequirement {
            doc /* The large EVs must run longer than 200km */
            attribute :>> requiredRange = 200[km];
        }

        analysis rangeAnalysisLarge :> largeEVAnalysis : RangeAnalysis {
            :>> simCond { :>> toTime = 5000[s]; }
            requirement :>> rangeRequirement = rangeRequirementLarge;
            return simulatedRange = vehicle.vehicleBehavior.output.distance {
                @MgS::SimRet {
                    ret = MgS::Retrieval::FINAL;
                }
            }
        }

        requirement <L2> efficiencyRequirementLarge :> largeEVRequirement : EfficiencyRequirement {
            doc /* The target efficiency of large EVs is 0.8. */
            attribute :>> requiredEfficiency = 0.8;
        }

        analysis efficiencyAnalysisLarge :> largeEVAnalysis : EfficiencyAnalysis {
            :>> simCond { :>> toTime = 1800[s]; }
            requirement :>> efficiencyRequirement = efficiencyRequirementLarge;

            return simulatedEfficiency = vehicle.efficiency { @MgS::SimRet; }
        }

        requirement <L3> maxSpeedRequirementLarge :> largeEVRequirement : MaxSpeedRequirement {
            doc /* The target maximum speed of large EVs is 140 [km/h]. */
            attribute :>> requiredMaxSpeed = 140 [km/h];
        }

        analysis maxSpeedAnalysisLarge :> largeEVAnalysis : MaxSpeedAnalysis {
            :>> simCond { :>> toTime = 700[s]; }
            requirement :>> maxSpeedRequirement = maxSpeedRequirementLarge;
            out voltage = vehicle.battery.output.voltage {
                @MgS::SimRet {
                    ret = Retrieval::FULL;
                }
            }
            return simulatedMaxSpeed = vehicle.vehicleBehavior.output.velocity {
                @MgS::SimRet {
                    ret = Retrieval::MAX;
                }
            }
        }
    }
}

Package LargeEV (c732bd99-4bfc-4ea3-8627-95129b72569d)


In [ ]:
%viz --view action --style lr --style stdcolor LargeEV::vehicle_large

## Analysis Evaluation

In [4]:
%mgs list

analysisCases, smallEVAnalysis, efficiencyAnalysisSmall, largeEVAnalysis, maxSpeedAnalysisSmall, rangeAnalysisSmall, self, rangeAnalysisLarge, subAnalysisCases, efficiencyAnalysisLarge, maxSpeedAnalysisLarge

In [5]:
%mgs exec rangeAnalysisSmall

Launching MATLAB...


Flow connection: mt2tt has no corresponding Simulink line
Flow connection: to2mf has no corresponding Simulink line
Flow connection: bv2pv has no corresponding Simulink line
Flow connection: pc2bc has no corresponding Simulink line
Flow connection: va2pa has no corresponding Simulink line
Flow connection: pf2vf has no corresponding Simulink line
Line (va2ta) |vehicleBehavior/accel| --> |EV2/Accel| does not correspond to any SysML element.
!!It had an MgS tag and was removed 
Line () |vehicleBehavior/distance| --> |EV2/Distance| does not correspond to any SysML element.
Line () |vehicleBehavior/velocity| --> |EV2/Velociy| does not correspond to any SysML element.
Block SLO:EV2/Accel[block] does not correspond to any SysML element.
Block SLO:EV2/Distance[block] does not correspond to any SysML element.
Block SLO:EV2/Velociy[block] does not correspond to any SysML element.
!Made a new line for connection: null 
!Made a new line for connection: null 
!Made a new line for connection: mt2tt 

In [7]:
%mgs exec maxSpeedAnalysisLarge

[Success] Binding: null is matched with Line () |powerTrain/voltage| --> |powerTrain/motor|
[Success] Binding: null is matched with Line (mc2bc) |powerTrain/motor| --> |powerTrain/current|
Flow connection: mt2rt has no corresponding Simulink line
Flow connection: rt2tt has no corresponding Simulink line
[Success] Flow connection: to2mf is matched with Line (to2mf) |powerTrain/tire| --> |powerTrain/motor|
[Success] Binding: null is matched with Line () |powerTrain/accel| --> |powerTrain/tire|
[Success] Binding: null is matched with Line (tf2vf) |powerTrain/tire| --> |powerTrain/force|
[Success] Flow connection: null is matched with Line (bv2pv) |EV2/battery| --> |EV2/voltage|
[Success] Flow connection: bv2pv is matched with Line (bv2pv) |EV2/battery| --> |powerTrain/voltage|
[Success] Flow connection: pc2bc is matched with Line (pc2bc) |powerTrain/current| --> |EV2/battery|
[Success] Flow connection: va2pa is matched with Line (va2pa) |vehicleBehavior/accel| --> |powerTrain/accel|
[Succ

In [6]:
%mgx open EV2

Opened MgX template: .\EV2.xlsm

In [ ]:
%mgs exec maxSpeedAnalysisSmall

In [ ]:
%mgp open EV2